# Neurohackademy Hackathon: Extracting Emotion-ROI Based Tracks
## hcp-des-emotional-states-project
#### Author: Diana Hobbs, +Ariel Rokem, +ChatGPT/stackoverflow/etc
#### Date: August 2024

In [1]:
# import libraries
import os
import csv
import dipy
import abagen 
import pandas as pd
import nibabel as nib

from dipy.io.streamline import load_trk
from utilities import ls, crawl
from pathlib import Path
from cloudpathlib import S3Path, S3Client

In [2]:
# get to base 
!cd
!ls

DTI_DF.csv                           credentials
DTI_DF_cleaned.csv                   nh2024-hcp1200-ml
HCP_DTI-Group_Copy.ipynb             structural_data.R
HCP_DTI.ipynb                        structural_neuroimaging.csv
HCP_data_structural_from_Melissa.csv subjects.csv
__pycache__                          utilities.py


In [111]:
# create .aws/credentials with access key ID and secret access key
!mkdir -p ~/.aws
!cd .aws
!touch credentials

# from NH course 
!echo '[hcp]' > ~/.aws/credentials
!echo 'aws_access_key_id = <<INPUT YOUR ACCESS KEY ID HERE>>' >> ~/.aws/credentials
!echo 'aws_secret_access_key = <<INPUT YOUR SECRET ACCESS KEY HERE>>' >> ~/.aws/credentials
!echo ' ' = '   ' >> ~/.aws/credentials # **MUST add a blank line after or it gets wonky

In [112]:
# from Ariel & NH course
# Make sure that we have a cache path:
cache_path = Path('/tmp/cache')
# Just because we have made a cache path object doesn't mean that the directory
# we made exists; here we check if it exists and make the directory if not.
if not cache_path.exists():
    cache_path.mkdir()

client = S3Client(
    local_cache_dir=cache_path,
    no_sign_request=True)

hcp_base_path = S3Path(
    "s3://open-neurodata", # **NOTE: we are using open-neurodata, not hcp-openaccess
    client=client)
hcp_subdirs = list(hcp_base_path.iterdir())
# There are about 1200 of these, so we won't show them all, just the first 10:
hcp_subdirs[:10]

[S3Path('s3://open-neurodata/ABagemouse'),
 S3Path('s3://open-neurodata/ARA'),
 S3Path('s3://open-neurodata/ailey'),
 S3Path('s3://open-neurodata/allan_johnson'),
 S3Path('s3://open-neurodata/ara_2016'),
 S3Path('s3://open-neurodata/bhatla'),
 S3Path('s3://open-neurodata/bigbrain'),
 S3Path('s3://open-neurodata/bloss'),
 S3Path('s3://open-neurodata/bock11'),
 S3Path('s3://open-neurodata/brainlit')]

In [113]:
# navigate to connectomes & extract DKT parcellated tracts
crawl(hcp_base_path / "hcp1200" / "sub-100206" / "ses-1" / "connectomes" )
crawl(hcp_base_path / "hcp1200" / "sub-100206" / "ses-1" / "connectomes" / "DKT_space-MNI152NLin6_res-1x1x1")

connectomes
   AAL_space-MNI152NLin6_res-1x1x1
      sub-100206_ses-1_run-1_dwi_AAL_space-MNI152NLin6_res-1x1x1_connectome.csv
   CPAC200_space-MNI152NLin6_res-1x1x1
      sub-100206_ses-1_run-1_dwi_CPAC200_space-MNI152NLin6_res-1x1x1_connectome.csv
   DKT_space-MNI152NLin6_res-1x1x1
      sub-100206_ses-1_run-1_dwi_DKT_space-MNI152NLin6_res-1x1x1_connectome.csv
   Desikan_space-MNI152NLin6_res-1x1x1
      sub-100206_ses-1_run-1_dwi_Desikan_space-MNI152NLin6_res-1x1x1_connectome.csv
   Glasser_space-MNI152NLin6_res-1x1x1
      sub-100206_ses-1_run-1_dwi_Glasser_space-MNI152NLin6_res-1x1x1_connectome.csv
   Schaefer1000_space-MNI152NLin6_res-1x1x1
      sub-100206_ses-1_run-1_dwi_Schaefer1000_space-MNI152NLin6_res-1x1x1_connectome.csv
   Schaefer200_space-MNI152NLin6_res-1x1x1
      sub-100206_ses-1_run-1_dwi_Schaefer200_space-MNI152NLin6_res-1x1x1_connectome.csv
   Schaefer300_space-MNI152NLin6_res-1x1x1
      sub-100206_ses-1_run-1_dwi_Schaefer300_space-MNI152NLin6_res-1x1x1_connectom

## Loading DKT parcellation labels

In [114]:
# get dkt ROI labels
dkt = abagen.datasets.fetch_desikan_killiany()
dkt_image = dkt["image"]
dkt_info = dkt["info"]

# load labels
labels = pd.read_csv(dkt_info, sep=",")
print(labels.shape)
labels.index=range(1, len(labels)+1)
print(labels)

# concatenate label/hemi/structure for ease of use
labels['hemi_label'] = labels['hemisphere'] + "_" + labels['label'] + "_" + labels['structure']

# select id / hemi_label
hemi_labels = labels.iloc[:, [0, 4]]
print(hemi_labels)

(83, 4)
    id                     label hemisphere            structure
1    1                  bankssts          L               cortex
2    2   caudalanteriorcingulate          L               cortex
3    3       caudalmiddlefrontal          L               cortex
4    4                    cuneus          L               cortex
5    5                entorhinal          L               cortex
6    6                  fusiform          L               cortex
7    7          inferiorparietal          L               cortex
8    8          inferiortemporal          L               cortex
9    9          isthmuscingulate          L               cortex
10  10          lateraloccipital          L               cortex
11  11      lateralorbitofrontal          L               cortex
12  12                   lingual          L               cortex
13  13       medialorbitofrontal          L               cortex
14  14            middletemporal          L               cortex
15  15           

In [115]:
# chatCPT extraordinaire 
# Set display options to show all rows and columns
pd.set_option('display.max_rows', None)   # None means no limit
pd.set_option('display.max_columns', None) # None means no limit
pd.set_option('display.expand_frame_repr', False) # Prevent frame expansion

In [116]:
print(hemi_labels["hemi_label"].all)

<bound method NDFrame._add_numeric_operations.<locals>.all of 1                        L_bankssts_cortex
2         L_caudalanteriorcingulate_cortex
3             L_caudalmiddlefrontal_cortex
4                          L_cuneus_cortex
5                      L_entorhinal_cortex
6                        L_fusiform_cortex
7                L_inferiorparietal_cortex
8                L_inferiortemporal_cortex
9                L_isthmuscingulate_cortex
10               L_lateraloccipital_cortex
11           L_lateralorbitofrontal_cortex
12                        L_lingual_cortex
13            L_medialorbitofrontal_cortex
14                 L_middletemporal_cortex
15                L_parahippocampal_cortex
16                    L_paracentral_cortex
17                L_parsopercularis_cortex
18                  L_parsorbitalis_cortex
19               L_parstriangularis_cortex
20                  L_pericalcarine_cortex
21                    L_postcentral_cortex
22             L_posteriorcingulate

In [117]:
# isolate roi labels

# caudalanteriorcingulate L: 2
# caudalanteriorcingulate R: 43
# rostralanteriorcingulate L: 25
# rostralanteriorcingulate R: 66
# lateralorbitofrontal L:11
# lateralorbitofrontal R:52
# medialorbitofrontal L:13
# medialorbitofrontal R:54
# amygdala L: 41
# amygdala R: 82
# thalamus L: 35
# thalamus R: 76
# hippocampus L: 40
# hippocampus R: 81

# selected_rois =  [2,43,25,66,11,52,13,54,41,82] ## initially amy<>orbitofronal amy<>ACC
# selected_rois =  [2,43,25,66,11,52,13,54,41,82,40,81,35,76] # added in hippocampus and thalamus
selected_rois =  [25,66,41,82,40,81,35,76] # removed orbitofrontal & caudal ACC

# 
selected_roi_labels = hemi_labels[hemi_labels['id'].isin(selected_rois)]
print(selected_roi_labels)

## initially amy<>orbitofronal amy<>ACC
# region_dict = {  2: "L_caudalanteriorcingulate_cortex",
#                 43: "R_caudalanteriorcingulate_cortex",
#                 25: "L_rostralanteriorcingulate_cortex",
#                 66: "R_rostralanteriorcingulate_cortex",
#                 11: "L_lateralorbitofrontal_cortex", 
#                 52: "R_lateralorbitofrontal_cortex", 
#                 13: "L_medialorbitofrontal_cortex",
#                 54: "R_medialorbitofrontal_cortex",
#                 41: "L_amygdala",
#                 82: "R_amygdala"  }

# added in hippocampus and thalamus
# region_dict = {  2: "L_caudalanteriorcingulate_cortex",
#                 43: "R_caudalanteriorcingulate_cortex",
#                 25: "L_rostralanteriorcingulate_cortex",
#                 66: "R_rostralanteriorcingulate_cortex",
#                 11: "L_lateralorbitofrontal_cortex", 
#                 52: "R_lateralorbitofrontal_cortex", 
#                 13: "L_medialorbitofrontal_cortex",
#                 54: "R_medialorbitofrontal_cortex",
#                 41: "L_amygdala",
#                 82: "R_amygdala",
#                 40: "L_hippocampus",
#                 81: "R_hippocampus",
#                 35: "L_thalamus",
#                 76: "R_thalamus"}

# removed orbitofrontal & caudal ACC
region_dict = { 
                25: "L_rostralanteriorcingulate_cortex",
                66: "R_rostralanteriorcingulate_cortex",
                41: "L_amygdala",
                82: "R_amygdala",
                40: "L_hippocampus",
                81: "R_hippocampus",
                35: "L_thalamus",
                76: "R_thalamus"}

print(region_dict)

    id                            hemi_label
25  25     L_rostralanteriorcingulate_cortex
35  35  L_thalamusproper_subcortex/brainstem
40  40     L_hippocampus_subcortex/brainstem
41  41        L_amygdala_subcortex/brainstem
66  66     R_rostralanteriorcingulate_cortex
76  76  R_thalamusproper_subcortex/brainstem
81  81     R_hippocampus_subcortex/brainstem
82  82        R_amygdala_subcortex/brainstem
{25: 'L_rostralanteriorcingulate_cortex', 66: 'R_rostralanteriorcingulate_cortex', 41: 'L_amygdala', 82: 'R_amygdala', 40: 'L_hippocampus', 81: 'R_hippocampus', 35: 'L_thalamus', 76: 'R_thalamus'}


In [130]:
# Define the data for columns 0 and 1: col 0 always has smaller id of the pair, col 1 always has larger id of the pair

## initially amy<>orbitofronal amy<>ACC
# col0 = [2, 25, 11, 13, 41, 41, 41, 41, 2, 25, 11, 13, 43, 66, 52, 54]  
# col1 = [41, 41, 41, 41, 43, 66, 52, 54, 82, 82, 82, 82, 82, 82, 82, 82] 

# added in hippocampus and thalamus
# col0 = [2, 25, 11, 13, 41, 41, 41, 41, 2, 25, 11, 13, 43, 66, 52, 54,
#        40, 81, 41, 41, 35, 35, 40, 76, 25, 25, 35, 66]  
# col1 = [41, 41, 41, 41, 43, 66, 52, 54, 82, 82, 82, 82, 82, 82, 82, 82,
#        41, 82, 81, 82, 40, 81, 76, 81, 35, 76, 66, 76] 

# removed orbitofrontal & caudal ACC
col0 = [25, 25, 41, 66, 40, 40, 41, 81, 
       25, 25, 35, 66, 35, 35, 40, 76]

col1 = [41, 82, 66, 82, 41, 82, 81, 82, 
       35, 76, 66, 76, 40, 81, 76, 81] 


# Create the DataFrame with columns 0 and 1
emo_roi_combo = pd.DataFrame({
    '0': col0,
    '1': col1
})
# combining ROI ids to make it easier to identify in next step
emo_roi_combo['combination'] = list(zip(emo_roi_combo.iloc[:, 0], emo_roi_combo.iloc[:, 1]))

print(emo_roi_combo)

     0   1 combination
0   25  41    (25, 41)
1   25  82    (25, 82)
2   41  66    (41, 66)
3   66  82    (66, 82)
4   40  41    (40, 41)
5   40  82    (40, 82)
6   41  81    (41, 81)
7   81  82    (81, 82)
8   25  35    (25, 35)
9   25  76    (25, 76)
10  35  66    (35, 66)
11  66  76    (66, 76)
12  35  40    (35, 40)
13  35  81    (35, 81)
14  40  76    (40, 76)
15  76  81    (76, 81)


## Loading n=646 subject IDs, creating a dictionary to hold DFs for each subject, filtering only emo-related ROI pairings

In [131]:
# load subject IDs 
subjects646_csv = pd.read_csv("/Users/hobbsd/Documents/GitHub/WashU_Courses/Neurohacademy/group_project/subjects.csv")

# extract subject numbers to list (I suggest using 5 IDs to test  ex. [:5, 0])
subjects646 = subjects646_csv.iloc[:, 0].tolist() 
# print(len(subjects_ID))

In [132]:
# initialize dictionary to store DFs for each subject
filtered_dfs = {}

# loop through subjects, create DFs, store in above dictionary
for subject in subjects646:
    # create & read csv for each subject & rename 3rd col sub specific 
    csv_path =  hcp_base_path / "hcp1200" / f"sub-{subject}" / "ses-1" / "connectomes" / "DKT_space-MNI152NLin6_res-1x1x1" / f"sub-{subject}_ses-1_run-1_dwi_DKT_space-MNI152NLin6_res-1x1x1_connectome.csv"
    # Check if the file exists
    if not csv_path.exists():
        print(f"File not found for subject {subject}, skipping...")
        continue
    
    try:
        # Attempt to read the CSV file
        df = pd.read_csv(csv_path.fspath, sep=" ", header=None)
        df = df.rename(columns={df.columns[2]: f"{subject}_tracts"})
    except pd.errors.EmptyDataError:
        print(f"File for subject {subject} is empty, skipping...")
        continue
    
    # Create a combination of columns 0 and 1 in df for filtering (this is to compare to the combination made prior)
    df['combination'] = list(zip(df.iloc[:, 0], df.iloc[:, 1]))
    
    # Filter df to only keep rows where combination is in emo_roi_combo
    filtered_rois = df[df['combination'].isin(emo_roi_combo['combination'])]
    
    # Drop the 'combination' helper column after filtering
    filtered_rois = filtered_rois.drop(columns=['combination'])
    
    # Store the filtered DataFrame in the filtered_dfs dictionary
    filtered_dfs[subject] = filtered_rois

File for subject 111413 is empty, skipping...
File for subject 118932 is empty, skipping...
File not found for subject 123925, skipping...
File not found for subject 125525, skipping...
File not found for subject 128127, skipping...
File not found for subject 129129, skipping...
File for subject 144832 is empty, skipping...
File for subject 149741 is empty, skipping...
File for subject 157942 is empty, skipping...
File for subject 178243 is empty, skipping...
File not found for subject 188751, skipping...
File not found for subject 195041, skipping...
File not found for subject 204218, skipping...
File not found for subject 205119, skipping...
File for subject 205826 is empty, skipping...
File not found for subject 206525, skipping...
File not found for subject 206727, skipping...
File for subject 227432 is empty, skipping...
File for subject 245333 is empty, skipping...
File not found for subject 432332, skipping...
File not found for subject 433839, skipping...
File for subject 47976

In [133]:
#filtered_dfs 

## Extract tract column from each subject and save in a DF

In [134]:
# extract 1st DF from dictionary 
first_df = next(iter(filtered_dfs.values()))
# copy into final DF
final_df = first_df.iloc[:, :3].copy()
final_df

,0,1,100408_tracts
824,25,35,373.0
828,25,41,490.0
842,25,76,2.0
845,25,82,1.0
1001,35,40,1418.0
1112,40,41,1710.0
1133,40,76,5.0
1137,40,82,11.0
1671,66,76,16.0
1676,66,82,55.0


In [135]:
# chatCPT extraordinaire 
final_df.columns = [0, 1, f"{next(iter(filtered_dfs.keys()))}_tracts"]

# Loop through the dictionary and merge each DataFrame's tracts column into final_df
for subject, df in filtered_dfs.items():
    if subject != next(iter(filtered_dfs.keys())):  # Skip the first subject already in final_df
        # Prepare the DataFrame to merge
        merge_df = df.iloc[:, :2].copy()  # Take the first two columns (col0 and col1)
        merge_df[f"{subject}_tracts"] = df.iloc[:, 2]  # Add the tracts column with subject-specific name
        
        # Merge with final_df based on match to columns 0 and 1
        final_df = pd.merge(final_df, merge_df, on=[0, 1], how='outer')

final_df.head()

,0,1,100408_tracts,100610_tracts,101006_tracts,101107_tracts,101309_tracts,101915_tracts,102311_tracts,102513_tracts,102816_tracts,103111_tracts,103414_tracts,103515_tracts,103818_tracts,104012_tracts,104416_tracts,104820_tracts,105014_tracts,105115_tracts,105216_tracts,105923_tracts,106016_tracts,106319_tracts,106521_tracts,107018_tracts,107321_tracts,107422_tracts,107725_tracts,108121_tracts,108222_tracts,108323_tracts,108525_tracts,108828_tracts,109123_tracts,110411_tracts,111009_tracts,111514_tracts,111716_tracts,112516_tracts,112920_tracts,113215_tracts,113619_tracts,113922_tracts,114217_tracts,114419_tracts,114621_tracts,114823_tracts,115017_tracts,115320_tracts,115825_tracts,116524_tracts,116726_tracts,117122_tracts,117324_tracts,117930_tracts,118124_tracts,118225_tracts,118528_tracts,118730_tracts,120111_tracts,120212_tracts,120515_tracts,120717_tracts,121416_tracts,121618_tracts,121921_tracts,122317_tracts,122822_tracts,123117_tracts,123420_tracts,123723_tracts,124220_tracts,124624_tracts,124826_tracts,126325_tracts,126628_tracts,127226_tracts,127630_tracts,127832_tracts,127933_tracts,128632_tracts,128935_tracts,129028_tracts,129331_tracts,130013_tracts,130114_tracts,130316_tracts,130417_tracts,130518_tracts,130619_tracts,130821_tracts,131217_tracts,131419_tracts,131722_tracts,131924_tracts,132017_tracts,133019_tracts,133625_tracts,133827_tracts,133928_tracts,134425_tracts,134728_tracts,135225_tracts,135528_tracts,135730_tracts,135932_tracts,136227_tracts,136631_tracts,136732_tracts,136833_tracts,137229_tracts,137633_tracts,138130_tracts,138231_tracts,138534_tracts,139233_tracts,139637_tracts,139839_tracts,140117_tracts,140824_tracts,140925_tracts,141119_tracts,141422_tracts,141826_tracts,143325_tracts,144428_tracts,145632_tracts,145834_tracts,146331_tracts,146432_tracts,146937_tracts,147636_tracts,148032_tracts,148133_tracts,148335_tracts,148941_tracts,149236_tracts,149337_tracts,149539_tracts,149842_tracts,150625_tracts,150726_tracts,151223_tracts,151324_tracts,151425_tracts,151526_tracts,151627_tracts,151728_tracts,151829_tracts,152225_tracts,152831_tracts,153025_tracts,153429_tracts,154229_tracts,154431_tracts,154532_tracts,154734_tracts,154835_tracts,154936_tracts,155635_tracts,155938_tracts,156233_tracts,156334_tracts,156637_tracts,157336_tracts,157437_tracts,158035_tracts,158136_tracts,158338_tracts,158540_tracts,158843_tracts,159138_tracts,159239_tracts,159340_tracts,159441_tracts,160123_tracts,161327_tracts,161630_tracts,161731_tracts,162026_tracts,162228_tracts,162329_tracts,162733_tracts,162935_tracts,163129_tracts,163331_tracts,163836_tracts,164030_tracts,164131_tracts,164636_tracts,164939_tracts,165638_tracts,165941_tracts,166438_tracts,167036_tracts,168139_tracts,168240_tracts,168341_tracts,169040_tracts,169444_tracts,170631_tracts,171330_tracts,171532_tracts,171633_tracts,172029_tracts,172130_tracts,172332_tracts,172534_tracts,172938_tracts,173334_tracts,173435_tracts,173536_tracts,173637_tracts,173940_tracts,174437_tracts,174841_tracts,175237_tracts,175338_tracts,175439_tracts,175742_tracts,176037_tracts,176542_tracts,177241_tracts,177645_tracts,178142_tracts,178950_tracts,179245_tracts,179346_tracts,180129_tracts,180432_tracts,180735_tracts,180937_tracts,181232_tracts,181636_tracts,182436_tracts,182739_tracts,183034_tracts,185139_tracts,185341_tracts,185442_tracts,185947_tracts,186141_tracts,186444_tracts,187345_tracts,187547_tracts,187850_tracts,188347_tracts,188549_tracts,189349_tracts,189450_tracts,190031_tracts,191033_tracts,191235_tracts,191336_tracts,191841_tracts,192035_tracts,192540_tracts,192641_tracts,192843_tracts,193239_tracts,194140_tracts,194645_tracts,194746_tracts,194847_tracts,195445_tracts,195950_tracts,196144_tracts,196750_tracts,197348_tracts,198249_tracts,198350_tracts,198451_tracts,198653_tracts,198855_tracts,199150_tracts,199453_tracts,200008_tracts,200109_tracts,200311_tracts,200614_tracts,201111_tracts,201414_tracts,201515_tracts,201818_tracts,202113_tracts,203418_tracts,20392

In [136]:
# add in the labels for the regions
final_df[0] = final_df[0].map(region_dict)
final_df[1] = final_df[1].map(region_dict)

final_df.head()

,0,1,100408_tracts,100610_tracts,101006_tracts,101107_tracts,101309_tracts,101915_tracts,102311_tracts,102513_tracts,102816_tracts,103111_tracts,103414_tracts,103515_tracts,103818_tracts,104012_tracts,104416_tracts,104820_tracts,105014_tracts,105115_tracts,105216_tracts,105923_tracts,106016_tracts,106319_tracts,106521_tracts,107018_tracts,107321_tracts,107422_tracts,107725_tracts,108121_tracts,108222_tracts,108323_tracts,108525_tracts,108828_tracts,109123_tracts,110411_tracts,111009_tracts,111514_tracts,111716_tracts,112516_tracts,112920_tracts,113215_tracts,113619_tracts,113922_tracts,114217_tracts,114419_tracts,114621_tracts,114823_tracts,115017_tracts,115320_tracts,115825_tracts,116524_tracts,116726_tracts,117122_tracts,117324_tracts,117930_tracts,118124_tracts,118225_tracts,118528_tracts,118730_tracts,120111_tracts,120212_tracts,120515_tracts,120717_tracts,121416_tracts,121618_tracts,121921_tracts,122317_tracts,122822_tracts,123117_tracts,123420_tracts,123723_tracts,124220_tracts,124624_tracts,124826_tracts,126325_tracts,126628_tracts,127226_tracts,127630_tracts,127832_tracts,127933_tracts,128632_tracts,128935_tracts,129028_tracts,129331_tracts,130013_tracts,130114_tracts,130316_tracts,130417_tracts,130518_tracts,130619_tracts,130821_tracts,131217_tracts,131419_tracts,131722_tracts,131924_tracts,132017_tracts,133019_tracts,133625_tracts,133827_tracts,133928_tracts,134425_tracts,134728_tracts,135225_tracts,135528_tracts,135730_tracts,135932_tracts,136227_tracts,136631_tracts,136732_tracts,136833_tracts,137229_tracts,137633_tracts,138130_tracts,138231_tracts,138534_tracts,139233_tracts,139637_tracts,139839_tracts,140117_tracts,140824_tracts,140925_tracts,141119_tracts,141422_tracts,141826_tracts,143325_tracts,144428_tracts,145632_tracts,145834_tracts,146331_tracts,146432_tracts,146937_tracts,147636_tracts,148032_tracts,148133_tracts,148335_tracts,148941_tracts,149236_tracts,149337_tracts,149539_tracts,149842_tracts,150625_tracts,150726_tracts,151223_tracts,151324_tracts,151425_tracts,151526_tracts,151627_tracts,151728_tracts,151829_tracts,152225_tracts,152831_tracts,153025_tracts,153429_tracts,154229_tracts,154431_tracts,154532_tracts,154734_tracts,154835_tracts,154936_tracts,155635_tracts,155938_tracts,156233_tracts,156334_tracts,156637_tracts,157336_tracts,157437_tracts,158035_tracts,158136_tracts,158338_tracts,158540_tracts,158843_tracts,159138_tracts,159239_tracts,159340_tracts,159441_tracts,160123_tracts,161327_tracts,161630_tracts,161731_tracts,162026_tracts,162228_tracts,162329_tracts,162733_tracts,162935_tracts,163129_tracts,163331_tracts,163836_tracts,164030_tracts,164131_tracts,164636_tracts,164939_tracts,165638_tracts,165941_tracts,166438_tracts,167036_tracts,168139_tracts,168240_tracts,168341_tracts,169040_tracts,169444_tracts,170631_tracts,171330_tracts,171532_tracts,171633_tracts,172029_tracts,172130_tracts,172332_tracts,172534_tracts,172938_tracts,173334_tracts,173435_tracts,173536_tracts,173637_tracts,173940_tracts,174437_tracts,174841_tracts,175237_tracts,175338_tracts,175439_tracts,175742_tracts,176037_tracts,176542_tracts,177241_tracts,177645_tracts,178142_tracts,178950_tracts,179245_tracts,179346_tracts,180129_tracts,180432_tracts,180735_tracts,180937_tracts,181232_tracts,181636_tracts,182436_tracts,182739_tracts,183034_tracts,185139_tracts,185341_tracts,185442_tracts,185947_tracts,186141_tracts,186444_tracts,187345_tracts,187547_tracts,187850_tracts,188347_tracts,188549_tracts,189349_tracts,189450_tracts,190031_tracts,191033_tracts,191235_tracts,191336_tracts,191841_tracts,192035_tracts,192540_tracts,192641_tracts,192843_tracts,193239_tracts,194140_tracts,194645_tracts,194746_tracts,194847_tracts,195445_tracts,195950_tracts,196144_tracts,196750_tracts,197348_tracts,198249_tracts,198350_tracts,198451_tracts,198653_tracts,198855_tracts,199150_tracts,199453_tracts,200008_tracts,200109_tracts,200311_tracts,200614_tracts,201111_tracts,201414_tracts,201515_tracts,201818_tracts,202113_tracts,203418_tracts,20392

In [137]:
# print(final_df)

In [140]:
# save DF as CSV & transpose
output_csv_path = "/Users/hobbsd/Documents/GitHub/WashU_Courses/Neurohacademy/group_project/DTI_DF.csv"
final_df.T.to_csv(output_csv_path)